In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import random
import missingno as msno
#from google.colab import data_table
from tabulate import tabulate
import warnings
warnings.filterwarnings("ignore")
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import RobustScaler,MinMaxScaler,PowerTransformer,StandardScaler
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import RFECV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression

In [16]:
final_df = pd.read_csv('./final_data_imputed_20_01_2024.csv')
final_df

,Date,opening_price,highest_price,lowest_price,closing_price,transactions_in_blockchain,avg_block_size,sent_by_adress,avg_mining_difficulty,avg_hashrate,...,avg_transaction_value,median_transaction_value,tweets,google_trends,active_addresses,top100_to_total_percentage,avg_fee_to_reward,number_of_coins_in_circulation,miner_revenue,next_day_closing_price
0,2013-01-02,13.24,13.24,12.77,13.22,39280,114077,31809,2979637,2.280465e+13,...,650.617,14.514,8193.000000,1.497000,43104.0,"Axes(0.125,0.11;0.775x0.77)",0.835,"Axes(0.125,0.11;0.775x0.77)","Axes(0.125,0.11;0.775x0.77)",13.09
1,2013-01-03,13.12,13.19,12.98,13.09,42147,108023,38197,2979637,2.372489e+13,...,542.730,19.732,8193.000000,1.798000,51268.0,"Axes(0.125,0.11;0.775x0.77)",0.925,"Axes(0.125,0.11;0.775x0.77)","Axes(0.125,0.11;0.775x0.77)",13.19
2,2013-01-04,13.09,13.26,12.96,13.19,48436,141811,34990,2979637,2.260818e+13,...,632.431,11.384,8193.000000,1.841000,47341.0,"Axes(0.125,0.11;0.775x0.77)",1.000,"Axes(0.125,0.11;0.775x0.77)","Axes(0.125,0.11;0.775x0.77)",13.19
3,2013-01-05,13.12,13.24,12.84,13.19,39455,118240,38008,2979637,2.259070e+13,...,697.556,13.945,8193.000000,1.826000,53417.0,"Axes(0.125,0.11;0.775x0.77)",0.885,"Axes(0.125,0.11;0.775x0.77)","Axes(0.125,0.11;0.775x0.77)",13.15
4,2013-01-06,13.19,13.19,13.03,13.15,32881,100608,30021,2979637,2.076659e+13,...,436.105,17.618,8193.000000,2.187000,42211.0,"Axes(0.125,0.11;0.775x0.77)",0.957,"Axes(0.125,0.11;0.775x0.77)","Axes(0.125,0.11;0.775x0.77)",13.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4027,2024-01-15,41700.91,43332.79,41682.62,42492.35,408385,754220,375303,73197634206448,4.427672e+20,...,73720.000,30.199,203592.746000,10.443000,658906.0,"Axes(0.125,0.11;0.775x0.77)",14.149,"Axes(0.125,0.11;0.775x0.77)","Axes(0.125,0.11;0.775x0.77)",43131.47
4028,2024-01-16,42483.97,43569.31,42033.30,43131.47,469997,752060,391211,73197634206448,4.766308e+20,...,84100.000,0.234,203645.994121,9.137000,653810.0,"Axes(0.125,0.11;0.775x0.77)",13.728,"Axes(0.125,0.11;0.775x0.77)","Axes(0.125,0.11;0.775x0.77)",42741.63
4029,2024-01-17,43132.80,43185.47,42164.99,42741.63,449294,747345,441886,73197634206448,4.007983e+20,...,115218.000,56.659,203693.132959,8.050000,689721.0,"Axes(0.125,0.11;0.775x0.77)",12.154,"Axes(0.125,0.11;0.775x0.77)","Axes(0.125,0.11;0.775x0.77)",41292.17
4030,2024-01-18,42740.17,42879.39,40605.54,41292.17,483875,739462,449416,73197634206448,4.756212e+20,...,99128.000,87.060,203732.860584,8.920000,732999.0,"Axes(0.125,0.11;0.775x0.77)",12.027,"Axes(0.125,0.11;0.775x0.77)","Axes(0.125,0.11;0.775x0.77)",41629.02


This function is useful for understanding the importance of different features in a dataset and selecting the most significant ones for further analysis. It leverages visualization techniques to make feature importance information more accessible.

In [18]:
def feature_imp(df,feat_importance,n_select_features):
    feat_imp_df = pd.DataFrame(data = {"Feature Name": df.columns,"Feature Importance":feat_importance})

    feat_imp_df = feat_imp_df.sort_values("Feature Importance",ascending=False)[:n_select_features]
    
    # Create a bar plot to visualize the feature importance's
    fig,ax = plt.subplots(figsize=(14,12))
    ax = sns.barplot(x = "Feature Importance",y = "Feature Name",
                    data=feat_imp_df,palette="nipy_spectral")
    plt.title('Feature Importances')

    cmap = plt.get_cmap('nipy_spectral_r',n_select_features)
    minfi = min(feat_imp_df['Feature Importance'])
    maxfi = max(feat_imp_df['Feature Importance'])
    norm = mpl.colors.Normalize(vmin=minfi,vmax=maxfi)
    sm = plt.cm.ScalarMappable(cmap=cmap,norm=norm)
    sm.set_array([])
    
    # Show the color bar
    # plt.colorbar(sm,ticks=np.linspace(minfi,maxfi,n_select_features),
    #             boundaries=np.arange(minfi,maxfi+0.001,.0001))
    
    # Display the grid lines along the x-axis
    ax.xaxis.grid()
    # Show the plot
    plt.show()
    # Return the DataFrame with feature importance's and selected features from the original DataFrame
    return feat_imp_df,df[feat_imp_df['Feature Name']]

Data Scaling

In [26]:
# problem here!!!
X = final_df.drop(['Date','next_day_closing_price'],axis=1)

scaler = RobustScaler()
X_scaled = X.copy()
    
X_scaled[X.columns] = scaler.fit_transform(X[X.columns])

scaler = MinMaxScaler()
X_scaled[X.columns] =  scaler.fit_transform(X_scaled[X.columns])

print(X_scaled)


ValueError: could not convert string to float: 'Axes(0.125,0.11;0.775x0.77)'

In [5]:
y = final_df['next_day_closing_price']
rf = RandomForestRegressor(n_estimators=100,n_jobs=-1,bootstrap=True,
                           verbose=5,random_state=1)
rf.fit(X_scaled,y)
     

NameError: name 'X_scaled' is not defined